In [1]:
# Disable gurobi logging output for this notebook.
try:
    import gurobipy
    gurobipy.setParam("OutputFlag", 0)
except ImportError:
    pass

import logging
logging.getLogger("").setLevel("CRITICAL")

# Configure roadrunner to allow for more output rows
import roadrunner
roadrunner.Config.setValue(
    roadrunner.Config.MAX_OUTPUT_ROWS, 1e6)

from cobra.sampling import sample

import mass.test
from mass.thermo import ConcSolver, sample_concentrations
from mass.simulation import ensemble, generate_ensemble_of_models

# Load the model
reference_model = mass.test.create_test_model("Glycolysis")

Academic license - for non-commercial use only


In [2]:
for reaction in reference_model.reactions:
    flux = reaction.steady_state_flux
    reaction.bounds = sorted([
        round(flux * 0.5, 12), round(flux * 1.5, 12)])
    
flux_samples = sample(reference_model, n=10, seed=25)

In [3]:
conc_solver = ConcSolver(
    reference_model,
    excluded_metabolites=["h_c", "h2o_c"],
    equilibrium_reactions=["ADK1"],
    constraint_buffer=1e-7)

conc_solver.setup_sampling_problem(
    conc_percent_deviation=0.8,
    Keq_percent_deviation=0)

conc_samples = sample_concentrations(conc_solver, n=8, seed=25)

reactions_to_check_percs = [
    r.id for r in reference_model.reactions
    if r not in reference_model.boundary]

In [4]:
%%time
flux_models = ensemble.create_models_from_flux_data(
    reference_model, data=flux_samples)

conc_models = []
for ref_model in flux_models:
    conc_models += ensemble.create_models_from_concentration_data(
        ref_model, data=conc_samples)

positive, negative = ensemble.ensure_positive_percs(
    models=conc_models, reactions=reactions_to_check_percs, 
    update_values=True)

feasible, infeasible_0 = ensemble.ensure_steady_state(
    models=positive, strategy="simulate",
    update_values=True)

feasible, infeasible_1 = ensemble.ensure_steady_state(
    models=feasible, strategy="simulate",
    perturbations={"kf_ATPM": "kf_ATPM * 1.5"},
    update_values=False)

mass/simulation/simulation.py:820 UserWarning: Unable to find a steady state for one or more models. Check the log for more details.


CPU times: user 2min 26s, sys: 3.2 s, total: 2min 29s
Wall time: 3min 25s


In [5]:
print("Total models generated: {0}".format(
    len(conc_models)))
print("Infeasible, negative PERCs: {0}".format(
    len(negative)))
print("Infeasible, no steady state found: {0}".format(
    len(infeasible_0)))
print("Infeasible, no steady state with pertubration 1: {0}".format(
    len(infeasible_1)))

Total models generated: 80
Infeasible, negative PERCs: 0
Infeasible, no steady state found: 0
Infeasible, no steady state with pertubration 1: 5


In [6]:
%%time

feasible, infeasible = generate_ensemble_of_models(
    reference_model=reference_model, 
    flux_data=flux_samples,
    conc_data=conc_samples,
    ensure_positive_percs=reactions_to_check_percs,
    strategy="simulate",
    perturbations={"kf_ATPM": "kf_ATPM * 1.5"},
    return_infeasible=True)

mass/simulation/simulation.py:820 UserWarning: Unable to find a steady state for one or more models. Check the log for more details.


Total models generated: 80
Feasible: 75
Infeasible, negative PERCs: 0
Infeasible, no steady state found: 0
Infeasible, no steady state with pertubration 1: 5
CPU times: user 37.3 s, sys: 1.21 s, total: 38.5 s
Wall time: 56.2 s
